# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv("../output_data/cities.csv")
df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=df[["Lat","Lng"]]
weight=df["Humidity"]
fig=gmaps.figure()
heatlayer=gmaps.heatmap_layer(locations, weights=weight,dissipating=False,point_radius=1)
fig.add_layer(heatlayer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
citie_narrows_df=df.loc[(df["Max Temp"] < 80) & (df["Max Temp"] > 70) & (df["Wind Speed"] < 10) & (df["Cloudiness"]==0), :]
citie_narrows_df=df.dropna()
citie_narrows_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [6]:
len(citie_narrows_df)

544

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = citie_narrows_df[["City", "Country", "Lat", "Lng"]]
hotel_df.head()


,City,Country,Lat,Lng
0,ostrovnoy,RU,68.05,39.51
1,mahebourg,MU,-20.41,57.70
2,qaanaaq,GL,77.48,-69.36
3,zhuhai,CN,40.71,112.04
4,cape town,ZA,-33.93,18.42


In [8]:
hotel_df["Hotel Name"]=""
hotel_df.head(5)

/Users/vineetsikriyahoo.com/opt/anaconda2/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,


In [9]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     assemble url and make API request
# request = requests.get(base_url, params=params)
# response = request.json()
# display(response)

In [8]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    #print(lat, lng)
    
    
    # get hotel type from df
#     Hotel_type= row['Hotel Name']

#     # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    

    # assemble url and make API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    Hotel_type = requests.get(base_url, params=params)
    Hotel_type =Hotel_type.json()
#     print(Hotel_type)
    
#     print(f"Retrieving Results for Index {index}: {Hotel_name}.")
#     response = requests.get(base_url, params=params).json()
#     
    try:
        hotel_df.loc[index, "Hotel_name"] = Hotel_type["results"][0]["name"]
        
        print(Hotel_type["results"][0]["name"])
    except(KeyError,IndexError):
        print("hotel not found")

hotel_df.head()

hotel not found


/Users/vineetsikriyahoo.com/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/vineetsikriyahoo.com/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Shandrani Beachcomber Resort & Spa
Qaanaaq Hotel
hotel not found
Southern Sun Waterfront Cape Town
Villa Enata
Shellharbour Resort & Conference Centre
Umka
The recreation center "Pushkinogorye"
Hotel Astoria
Hotel Esmeralda
Paradise Inn
hotel not found
Hotel Posada San Agustin
Hilton Albany
Gostinitsa Viliuisk
Albatros Hotel
Casa Tia Villa
Art-Otel' " Kupecheskiy"
Magnum Engineering & Construction
Observatory Guest House
Hotel de la Plage - Damgan - Morbihan - Bretagne
Salinas do Maragogi All Inclusive Resort
hotel not found
Reng Ta Chai Boutique HomeStay
जय मां दुर्गा मंदिर
Hotel Dreams Del Estrecho
Pension Maro'i
The Shingles Riverside Cottages
Old Observatory Youth Hostel Qingdao
Airport Inn
hotel not found
hotel not found
Long House Hotel Bethel
More Stay Inn
Dancenter Gurihuset
İç Yenice Köy Konağı
hotel not found
Misty Waves Boutique Hotel
See The Sea
The Explorer Hotel
Oyster Bay House Rental
Finch Bay Galapagos Hotel
Hotel El Chele Romero
hotel not found
HAMMER RUN
The Halyards

The Ritz-Carlton, Half Moon Bay
Mantra Nelson Bay
Ron and Tony's Bed & Breakfast
Northern Light Inn & Max's Restaurant
Super Leisure Hotel
hotel not found
HM Sams Lodge
Melrose
hotel not found
محمد بلال عبدالله البلوشي
Tiara Labuan Hotel
Econo Lodge
Energetik, Pansionat Ooo "Grand Baykal"
Restaurante e Pousada Boquira
Atlantis Hotel
Living Soul Garden Hotel
Radisson Blu Polar Hotel, Spitsbergen
Hotel Relax Inn
Cafe Forsaz
Balochistan Hotel
hotel not found
Bayaz Han (Beyaz Han)
Hotel Les Rochers
Ahipara Holiday Park
Backpackers Inn on the Beach
Volcano Sea View Tree House
Hôtel Château de la Tour
Oao "Susumanzoloto"
Hotel Zum Grünen Baum Plattling
Mato Grosso Hotel
Hotel Galvarino
Elk County Elk Farm
Omahu Motor Lodge
Prozhivaniye V Okhotske
Hotel Heilmann Lyberth
Kost Pria
HOTEL LA ROCA
Hotel Gamboa
hotel not found
Tiara Labuan Hotel
Nhà Hàng Thiên Thanh - CT. TNHH Thiên Lộc
Gostinitsa Aray
Knobby's Fly-In Camps
T A Commercial Hotel
Residence 'I Delfini'
Busta House Hotel
Gostinitsa Iv

,City,Country,Lat,Lng,Hotel_name
0,ostrovnoy,RU,68.05,39.51,NaN
1,mahebourg,MU,-20.41,57.70,Shandrani Beachcomber Resort & Spa
2,qaanaaq,GL,77.48,-69.36,Qaanaaq Hotel
3,zhuhai,CN,40.71,112.04,NaN
4,cape town,ZA,-33.93,18.42,Southern Sun Waterfront Cape Town


In [9]:
hotel_df.to_csv("hotel_Output.csv")

hotel_df.head(5)

,City,Country,Lat,Lng,Hotel_name
0,ostrovnoy,RU,68.05,39.51,NaN
1,mahebourg,MU,-20.41,57.70,Shandrani Beachcomber Resort & Spa
2,qaanaaq,GL,77.48,-69.36,Qaanaaq Hotel
3,zhuhai,CN,40.71,112.04,NaN
4,cape town,ZA,-33.93,18.42,Southern Sun Waterfront Cape Town


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# for index, row in 
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig